https://python-visualization.github.io/folium/quickstart.html#Getting-Started


In [ ]:
import numpy as np
import folium
import awswrangler as wr
import pandas as pd
from datetime import timedelta

location_log = pd.read_parquet("location-log.snappy.parquet")
location_log = location_log[["latitude", "longitude", "location_accuracy", "location_time"]]
location_log = location_log[~location_log["latitude"].isna()]
location_log = location_log[~location_log["location_accuracy"].isna()]
location_log['location_time'] = pd.to_datetime(location_log['location_time'], unit='ms')
location_log['location_time'] = location_log['location_time'] + timedelta(minutes=330)
sample_user_50 = location_log[location_log["location_accuracy"] < 50].copy()
print(len(sample_user_50))

In [ ]:
# find color : 새벽 2시이면 파란색, 낮 2시이면 빨간색
#
time_color_ratio = sample_user_50.apply(lambda r: 1 - abs((((r["location_time"].hour - 2 + 24) % 24) - 12) / 12), axis='columns')
red_color = time_color_ratio.apply(lambda r: hex(int(r * 255))[2:])
red_color = red_color.apply(lambda r: ("0" + r) if len(r) == 1 else r )
blue_color = time_color_ratio.apply(lambda r: hex(255 - int(r * 255))[2:])
blue_color = blue_color.apply(lambda r: ("0" + r) if len(r) == 1 else r )
sample_user_50["color"] = "#" + red_color + "00" + blue_color


In [2]:
from geopy import Point
from geopy import distance
import statistics
# clustering

class Location:

    def __init__(self, latitude, longitude, accuracy, time, color):
        self.time = time
        self.accuracy = accuracy
        self.longitude = longitude
        self.latitude = latitude
        self.color = color


class LocationCluster:
    centroid = None

    def __init__(self, location: Location):
        self.locations = []
        self.dates = set()
        self.night_time_dates = set()
        self.add_location(location)

    def add_location(self, location: Location):
        self.locations.append(location)
        self.update_centroid()
        self.dates.add(location.time.date())
        if (location.time.hour < 6) or (location.time.hour > 22):
            self.night_time_dates.add(location.time.date())

    def update_centroid(self):
        centroid_latitude = statistics.mean(map(lambda l: l.latitude, self.locations))
        centroid_longitude = statistics.mean(map(lambda l: l.longitude, self.locations))
        centroid_accuracy = max(map(lambda l: l.accuracy, self.locations))
        self.centroid = Location(centroid_latitude, centroid_longitude, centroid_accuracy, 0, "")

    def is_in_cluster(self, location: Location):
        d = distance.distance((location.latitude, location.longitude), (self.centroid.latitude, self.centroid.longitude)).meters
        if (d - self.centroid.accuracy - location.accuracy) < 0:
            return True
        return False

clusters = []



def add_in_clusters(location: Location):
    for c in clusters:
        if c.is_in_cluster(location):
            c.add_location(location)
            return True

    return False

for i in range(len(sample_user_50)):

    r = sample_user_50.iloc[i]
    l = Location(r["latitude"], r["longitude"], r["location_accuracy"], r["location_time"], r["color"])

    if not add_in_clusters(l):
        clusters.append(LocationCluster(l))

clusters.sort(key=lambda c: len(c.dates), reverse=True)
clusters_top5 = clusters[0:5]
clusters_top5.sort(key=lambda c: len(c.night_time_dates), reverse=True)

0        0825756651de060e
1196     0bfa10079bd11392
1835     15d566766e5fb440
2032     19edbdc7b07bef63
2385     260cb7bf1e5b56de
2481     27bee8cc6aef3d0a
2861     42d8bdbdde8a4156
3059     4446ccf2950b2992
3262     4929d07dcb1b1d14
3568     4d8733074e3834d6
3599     51d7f33b1f123b5c
3675     54ad187401f55f07
3721     55ca7a358d8e918c
3858     57eb0de6bb2dfac3
5195     5be78ea100322081
6619     5e767767c310ca70
8228     6233ca2d6cb00b85
9136     633d038dd2e2dd99
9208     6de40ef7f4a73ff8
9302     7df3e9cc17c7dd78
9695     7e26d12f1e3e45e9
9711     7e5ce961c546c586
9758     83e7d64adc914738
9984     91384d3e12c8dc6f
10249    97eb106f53401e6d
10342    9ede52ed89b11a20
11069    a5a88600c29e08ed
11202    b8f9f37729918b31
12302    bcb1ac06cba2a040
12523    bd333a24bd96ef5a
12855    c11a334ffc46b190
13351    c54c35fbea04bd1a
13548    c864575d4070c063
13592    cf0f39d4681c61d2
13893    d4c0d1fdb7dd538e
14044    de64d637a0249d7d
14252    e15ef93c28df56ca
14884    ec7e7eae92862ff0
17336    f38

In [5]:

def add_cluster(cluster: LocationCluster, first, map_):
    for l in cluster.locations:
        folium.Circle(
            location=[l.latitude, l.longitude], popup=f"time: {l.time}, <a href='https://www.google.com/maps/search/{l.latitude},+{l.longitude}/@{l.latitude},{l.longitude},17z'>google map</a>",
            color=l.color, fill=False,
            radius=l.accuracy).add_to(map2)

    folium.Marker(
        location=[cluster.centroid.latitude, cluster.centroid.longitude],
        tooltip=f"date count : {len(cluster.dates)}, night count : {len(cluster.night_time_dates)}",
        popup=f"<a href='https://www.google.com/maps/search/{cluster.centroid.latitude},+{cluster.centroid.longitude}/@{cluster.centroid.latitude},{cluster.centroid.longitude},17z'>google map</a>",
        icon=folium.Icon(color="red", icon="info-sign") if first else None
    ).add_to(map_)

latitude = clusters_top5[0].centroid.latitude
longitude = clusters_top5[0].centroid.longitude
location = [latitude, longitude]
map2 = folium.Map(location=location, tiles="cartodbpositron",
                  zoom_start=15)
for i in range(len(clusters_top5)):
    c = clusters_top5[i]
    add_cluster(c, i == 0, map2)
map2.save("test.html")


952


In [ ]:
bb = l.time.hour

In [60]:
bb = l.time.hour

In [ ]:



latitude = sample_user["latitude"].iloc[0]
longitude = sample_user["longitude"].iloc[0]
location = [latitude, longitude]

map2 = folium.Map(location=location, tiles="cartodbpositron",
                 zoom_start=15)

sample_user.apply(lambda row: folium.Circle(
    location=[row["latitude"], row["longitude"]], popup=row["time"],
    color=row["color"], fill=False,
    radius=row["location_accuracy"]).add_to(map2), axis=1)

map2

In [ ]:


sample_user_home = sample_user[(sample_user["time"] < 7) | (sample_user["time"] > 22)].copy()
latitude = sample_user_home["latitude"].iloc[0]
longitude = sample_user_home["longitude"].iloc[0]
location = [latitude, longitude]

map3 = folium.Map(location=location, tiles="cartodbpositron",
                 zoom_start=11)

sample_user_home.apply(lambda row: folium.Circle(
    location=[row["latitude"], row["longitude"]], popup=row["time"],
    color=row["color"], fill=False,
    radius=row["location_accuracy"]).add_to(map3), axis=1)

map3

In [ ]:
sample_user_home

In [ ]:
sample_user_acc = sample_user[(sample_user["location_accuracy"] <=20)].copy()
latitude = sample_user_acc["latitude"].iloc[0]
longitude = sample_user_acc["longitude"].iloc[0]
location = [latitude, longitude]

map4 = folium.Map(location=location, tiles="cartodbpositron",
                 zoom_start=11)

sample_user_acc.apply(lambda row: folium.Circle(
    location=[row["latitude"], row["longitude"]], popup=row["time"],
    color=row["color"], fill=False,
    radius=row["location_accuracy"]).add_to(map4), axis=1)

map4